In [ ]:
#mecab설치
!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
#ekonlpy 설치
!git clone https://github.com/entelecheia/eKoNLPy.git
!cd eKoNLPy
import os
os.chdir('/content/drive/MyDrive/miniproject/eKoNLPy')
!pip install .
!pip install . --upgrade (for upgrade)

# news 기사 n-gram화

In [ ]:
# news 기사 n-gram화

mpck = MPCK()

def make_ngram(df):
    try:
        # content(내용) 컬럼의 text N-gram화
        tokens=mpck.tokenize(df['content'])
        ngrams=mpck.ngramize(tokens)
        tokens_text=",".join(tokens)
        ngrams_text=",".join(ngrams)
        total_text=tokens_text+","+ngrams_text
    except Exception as e:
        print(e)
        total_text=""
    return total_text

df=pd.read_csv(r"뉴스기사content.csv")
mpck = MPCK()
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)
df['ngrams']=df.apply(make_ngram,axis=1)
df.to_csv(r"뉴스기사n-gram.csv", encoding='utf-8-sig', index=False)

# 데이터 병합

In [ ]:
import pandas as pd

In [ ]:
# .csv 파일들 데이터프레임 concat

import os
해당 경로의 디렉토리 이동
os.chdir('/content/drive/MyDrive/miniproject/mergy/mergy2')

import pandas as pd
list_files = os.listdir()
#채권분석 리포트, 의사록, 네이버 뉴스 파일 공통 컬럼 (['date','content']) concat
concat_df = pd.concat(map(pd.read_csv, list_files), ignore_index=True)

# 전체 파일에서 날짜 형식이 다른거 통일
for i in range(len(concat_df)):
    s = concat_df['date'][i]
    concat_df['date'][i] = s.replace('-','.')
concat_df.sort_values(by=['date'], axis=0, inplace=True)
concat_df

In [ ]:
#concat된 df에 콜금리의 labeling 컬럼 추가 (date 기준으로 merge)
df_call = pd.read_csv('/content/drive/MyDrive/miniproject/callfinal1.csv', encoding='utf-8')
merge_df = pd.merge(concat_df, df_call, on="date", how='right')
merge_df

In [ ]:
#필요없는 컬럼 정리, dropna, 중복값 정리
merge_df = merge_df.drop(['Unnamed: 0',  'Unnamed: 0.1'], axis=1)
merge_df = merge_df.drop(['callrate_x',  '한달전 date_x', '한달전 callrate_x', 'labeling_x', 'callrate_y', '한달전 date_y'], axis=1)
merge_df = merge_df.drop(['한달전 callrate_y'], axis=1)
merge_df

In [ ]:
#라벨링 이름 원래대로 변경
merge_df.rename(columns={'labeling_y':'labeling'}, inplace=True)
merge_df

In [ ]:
# 중복값 결측치 제거
merge_df = merge_df.dropna(axis=0)
merge_df = merge_df.drop_duplicates()
merge_df.to_csv('/content/drive/MyDrive/miniproject/merge_final_ver3.csv',encoding='utf-8-sig',index=False)


# 의사록 한글자 처리

In [ ]:
import pandas as pd
minutes = pd.read_csv('/content/drive/MyDrive/workspace/bok/minutes_data_save(08_21).csv',encoding='utf-8-sig')
minutes.sort_values(by='date',ascending=True,inplace=True)

import numpy as np

def remove_oneletter(text):
    ngram_list = []
    for j in range(len(text.split(','))):
        ngram = text.split(',')[j]
        ngram_list.append(ngram)
        letter = ngram.split('/')[0]
        letter2 = letter.split(';')[0]
        if len(letter) == 1 or len(letter2) == 1:
            # print(ngram)
            for i,v in enumerate(ngram_list) :
                if v == ngram:
                    ngram_list[i] = np.nan
    # print(ngram_list)
    removed = ','.join(ng for ng in ngram_list if ng is not np.nan)
        
    return removed

In [ ]:
from tqdm import tqdm

removed_text = []
for i in tqdm(range(len(minutes))):
    before_text = minutes['ngrams'][i]
    print(remove_oneletter(before_text))
    removed_text.append(remove_oneletter(before_text))

minutes['ngrams'] = removed_text
minutes.to_csv('ngrams_final_except1.csv',encoding='utf-8-sig',index=False)